In [15]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [16]:
dataset = "Dataset/simpsons_dataset"

img_size = 64
batch_size = 128
epochs = 30
input_shape = (img_size, img_size, 3) # 64 x 64 pixels with RGB

map_characters = {0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson',
                  3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel',
                  7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lisa_simpson',
                  11: 'marge_simpson', 12: 'milhouse_van_houten', 13: 'moe_szyslak',
                  14: 'ned_flanders', 15: 'nelson_muntz', 16: 'principal_skinner', 17: 'sideshow_bob'}

num_classes = len(map_characters)

In [17]:
def load_data():
    data = []
    labels = []
    for character_id, character_name in map_characters.items():
        character_folder = os.path.join(dataset, character_name)
        for img_file in os.listdir(character_folder):
            img_path = os.path.join(character_folder, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size)).astype('float32') / 255
            data.append(img)
            labels.append(character_id)
    data = np.array(data)
    labels = np.array(labels)
    return data, labels

data, labels = load_data()

In [18]:
total_samples = len(data)
iterations_per_epoch = int(round(total_samples / batch_size,0))
decay_steps = int(iterations_per_epoch * epochs)
initial_learning_rate = 1e-3

lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.9)

In [19]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

y_train = to_categorical(y_train, len(map_characters))
y_test = to_categorical(y_test, len(map_characters))

In [21]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(len(map_characters), activation='softmax'))

# opt = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
opt = Adam(learning_rate=lr_schedule)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train) / batch_size, epochs=epochs)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

model.save("modelV9.h5")

Epoch 1/30
118/118 [==============================] - 37s 301ms/step - loss: 4.5569 - accuracy: 0.1531
Epoch 2/30
118/118 [==============================] - 39s 325ms/step - loss: 2.8354 - accuracy: 0.2531
Epoch 3/30
118/118 [==============================] - 37s 313ms/step - loss: 2.3567 - accuracy: 0.3325
Epoch 4/30
118/118 [==============================] - 35s 298ms/step - loss: 2.0130 - accuracy: 0.4333
Epoch 5/30
118/118 [==============================] - 35s 297ms/step - loss: 1.7845 - accuracy: 0.5218
Epoch 6/30
118/118 [==============================] - 35s 293ms/step - loss: 1.5762 - accuracy: 0.5981
Epoch 7/30
118/118 [==============================] - 35s 294ms/step - loss: 1.4243 - accuracy: 0.6473
Epoch 8/30
118/118 [==============================] - 35s 298ms/step - loss: 1.3323 - accuracy: 0.6801
Epoch 9/30
118/118 [==============================] - 35s 295ms/step - loss: 1.2826 - accuracy: 0.7008
Epoch 10/30
118/118 [==============================] - 35s 297ms/step - l